# Route Optimization for Boston Tot Sprays

### Problem Description

"The Parks department wants to be able to regularly visit each tot spray to check if it is clean and fully functional. Please provide an algorithm that would optimize the total travel time between all the tot sprays. It does not have to be the shortest route, but rather an algorithm that would produce an ordered list of the various locations for the driver to visit that would cut down the time compared to them guessing the best route themselves."

### First Approaches

The task of visiting each Boston tot spray in minimum time is an application of the infamous "Traveling Salesman Problem".  The Parks department needs to visit each tot spray once, in succession, until all locations have been hit.  

The route optimization here presented is not an exact solution to the problem, but rather an iterative approach that can be used to improve a given route.  Alternative algorithms and the thought process behind my selection can be found toward the end of this notebook.

#### Terminology

Each tot spray is a *node*, the path linking each node an *edge*, and each route used to visit all of them a *tour*.  Each node is connected to every other node by an edge, which carries a *weight* given by the distance between the nodes.  The goal is to choose the optimal sequence of edges to make up the shortest tour.

### The 2-Opt Algorithm

The fundamental idea behind an iterative approach to optimization is to make a small change and see if the result improves.  For our purposes this means that we start with a randomly chosen route, alter the route slightly, and determine if we get a shorter tour length.  If so, we remember the better tour, do another route alteration, rinse and repeat.

The 2-opt algorithm chooses an edge and reverses its direction.  (This is the same as choosing two nodes and swapping them).  If the reversal improves the tour length, it's kept, if not, the original tour is kept.  This process is repeated by looping over pairs of nodes until all pairs have been considered.

A => B => C => D becomes A => C => B => D, if ABCD > ACBD <br>
A => C => B => D becomes A => C => D => B, if ACBD > ACDB <br>
And so on.

The concept of 2-opt is to "untangle" crossing routes over time.  With many iterations the route becomes smoother and smoother, optimizing the tour length.

***

Let's introduce the code:

### Code Preparation

Import libraries and dataset.  I used random sampling on a smaller number of nodes for troubleshooting.  I add an entry to the dataframe to include City Hall.

In [64]:
import pandas as pd
import numpy as np
from random import shuffle, sample, randint

#for random sampling
n = 49 #number of records in file
s = 20 #desired sample size
skip = sorted(sample(range(1,n+1),n-s))

#input path to the csv file here:
ts_df = pd.read_csv("/Users/mnksmith/Documents/Job Applications/Tot_Sprays.csv")#, skiprows=skip)

#add City Hall
home_lat, home_lon = 42.360644, -71.058014
ts_df.loc[-1] = [home_lon, home_lat, 0, 0, 'Central Boston', 'City Hall', '1 City Hall Square', 'TRUE', '10', '1 City Hall Square, Boston, MA 02201', 'Address', 0, 0, 0, 'None']  # adding a row
ts_df.index = ts_df.index + 1  # shifting index
ts_df.sort_index(inplace=True)

What does a row of data look like?

In [68]:
ts_df.head(3)

,X,Y,FID,OBJECTID,Neighborho,Park_Name,Address_Te,Matched,MatchScore,MatchText,MatchType,MatchId,MatchXCoor,MatchYCoor,Match_Code
0,-71.058014,42.360644,0,0.0,Central Boston,City Hall,1 City Hall Square,TRUE,10,"1 City Hall Square, Boston, MA 02201",Address,0.0,0.0000,0.000,None
1,-71.165966,42.351591,1,1.0,Allston/Brighton,Hardiman Park,380 Faneuil Street,TRUE,10,"380 Faneuil St, Brighton, MA 02135",Address,342467.0,746458.8680,2953325.543,"STREET_EXACT, ADDRESS_EXACT, XY_IMPROVEMENT"
2,-71.161191,42.355456,2,2.0,Allston/Brighton,Hobart Park,85 Hobart Street,TRUE,10,"85 Hobart St, Brighton, MA 02135",Address,342478.0,747744.0763,2954739.153,"STREET_EXACT, ADDRESS_EXACT, XY_IMPROVEMENT"


We're just interested in the lat/lon coordinates stored in columns X and Y.

Next let's create a function to calculate the distance between two nodes and a function to give the total length of the entire tour.  The distance is calculated by taking the coordinates of each node and applying the haversine formula .

In [13]:
def distance(node1, node2, earth_radius=6371):
    lat1, lon1, lat2, lon2 = ts_df.Y[node1], ts_df.X[node1], ts_df.Y[node2], ts_df.X[node2]
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

def calc_length(tour):
    length = 0
    for i in range(len(tour)):
        length += distance(tour[i-1], tour[i])

    return length

Now let's write the 2-opt algorithm.  A few things to note:
 - To make the tour start and finish at City Hall, I appended a final node to the tour and excluded the first and last nodes from consideration in the swapping process.
 - The nested for-loops consider edges for potential reversal and iterate.
 - This process may produce a different end route given different starting conditions, so the algorithm repeats `N_ITER` times, shuffling the inital tour order each time.

In [5]:
def two_opt(ts_df):
    best_tour = []
    best_length = float('inf')
    N_ITER = 15
    

    for i in range(N_ITER):
        tour = list(range(len(ts_df.index)))
        shuffle(tour)
        tour.append(0)
        length = calc_length(tour)

        changed = True
        while changed:

            changed = False

            for a in range(0, len(ts_df.index)):

                for b in range(a+1, len(ts_df.index)-1):

                    new_tour = tour[:a] + tour[a:b][::-1] + tour[b:]
                    new_length = calc_length(new_tour)

                    if new_length < length:
                        length = new_length
                        tour = new_tour
                        changed = True

        if length < best_length:
            best_length = length
            best_tour = tour

    return best_tour, best_length

### Results

The two-opt function ouputs an ordered path and a tour length.

In [86]:
path, length = two_opt(ts_df)
print('Path: ', path)
print('Length: %.2f km' % (length))

Path:  [0, 7, 19, 11, 18, 9, 10, 6, 5, 42, 41, 3, 4, 2, 1, 43, 49, 33, 20, 21, 24, 23, 15, 13, 47, 22, 44, 14, 16, 25, 29, 46, 45, 35, 28, 34, 31, 32, 30, 36, 37, 27, 26, 12, 17, 39, 48, 40, 38, 8, 0]
Length: 61.89 km


Converting the path indices to the park names and re-ordering the original dataframe to the new route:

In [72]:
output_route = [0] * 51
output_df = ts_df.copy()
#output_df.append(pd.Series([np.nan]), ignore_index = True)
for i in range(len(path)-1):
    output_route[i] = ts_df['Park_Name'][path[i]]
    output_df.iloc[i] = ts_df.iloc[path[i]]
output_route[50] = ts_df['Park_Name'][path[50]]


print(output_route)
output_df.head(5)

['City Hall', 'Boston Common Frog Pond', 'Boston Common Tot Lot', 'Christopher Columbus Park', 'Puopolo Playground', 'Rutherford Union Playground', 'Doherty Playground', 'East Boston Memorial Park', 'Byrne Playground', 'Sumner/Lamson Street Playground', 'Medal of Honor Park', 'Buckley Playground', 'Sweeney Playground', 'Ringgold Park', 'Hooker/Sorento Playground', 'Ringer Playground', 'Hobart\xa0Park', 'Hardiman Park', 'Hynes Playground', 'Healy Playground', 'Malcolm X Park', 'McGann Playground\xa0', 'Ernest Chery Jr. Playground', 'Walker Playground', 'Thetford/Evans Park', 'Roberts Park', 'Rev. Loesch Park', 'Elmhurst Park', 'Harambee Park (Franklin Field)', 'Erie/Ellington Street Playground', 'Ripley Playground', 'Ronan Park', 'Quincy/Stanley Playground', "Children's Playground", 'Holborn Street Playground', 'Laviscount Park', 'Trotter Playground', 'Father Roussin Park', 'El Parquesito Dela Hermandad, Franklin Park', 'Marcella/Connolly Park', 'Jeep Jones Park', 'Lambert Avenue Playgr

,X,Y,FID,OBJECTID,Neighborho,Park_Name,Address_Te,Matched,MatchScore,MatchText,MatchType,MatchId,MatchXCoor,MatchYCoor,Match_Code
0,-71.058014,42.360644,0,0.0,Central Boston,City Hall,1 City Hall Square,TRUE,10,"1 City Hall Square, Boston, MA 02201",Address,0.0,0.0000,0.000,None
1,-71.066340,42.357160,6,7.0,Back Bay/Beacon Hill,Boston Common Frog Pond,40 Beacon Street,TRUE,10,"40 Beacon St, Boston, MA 02108",Address,11357.0,773378.2413,2955476.161,"STREET_EXACT, ADDRESS_EXACT, XY_IMPROVEMENT"
2,-71.063827,42.355427,5,6.0,Back Bay/Beacon Hill,Boston Common Tot Lot,139 Tremont Street,TRUE,10,"139 Tremont St, Boston, MA 02108",Address,341542.0,774060.7756,2954848.024,"STREET_EXACT, ADDRESS_EXACT, XY_IMPROVEMENT"
3,-71.052183,42.360995,7,8.0,Central Boston,Christopher Columbus Park,110 Atlantic Avenue,TRUE,10,"110 Atlantic Ave, Boston, MA 02110",Address,161702.0,777197.2897,2956893.563,"STREET_EXACT, ADDRESS_EXACT, XY_IMPROVEMENT"
4,-71.053919,42.368277,8,9.0,Central Boston,Puopolo Playground,517 Commercial Street,TRUE,10,"517 Commercial St, Boston, MA 02109",Address,341654.0,776714.3352,2959544.900,"STREET_EXACT, ADDRESS_EXACT, XY_IMPROVEMENT"


To check if the route produced by this algorithm makes sense, let's plot the route on top of a map of boston.  The gmplot package is necessary to run this section of the code (`pip install gmplot`).

In [92]:
from gmplot import gmplot
from IPython.display import IFrame
from IPython.core.display import HTML 


gmap = gmplot.GoogleMapPlotter(42.35, -71.064361, 12)

ts_lats = np.empty(len(ts_df.index))
ts_lons = np.empty(len(ts_df.index))
for i in range(0,len(ts_df.index)):
    ts_lats[i], ts_lons[i] = ts_df.Y[i], ts_df.X[i]
gmap.scatter(ts_lats, ts_lons, c='red', symbol='.', size=200, marker=False)

for j in range(0,len(path)-1):
    path_lats, path_lons = zip(*[(ts_lats[path[j]],ts_lons[path[j]]), (ts_lats[path[j+1]],ts_lons[path[j+1]])])
    gmap.plot(path_lats, path_lons, 'darkblue', edge_width=5)

ch_lat, ch_lon = zip(*[(ts_df.Y[0], ts_df.X[0])])
gmap.scatter(ch_lat, ch_lon,'red')

gmap.draw("totspray_map3.html")

IFrame("totspray_map3.html", width=750, height=700)

The mapped route passes the sanity check--common sense says that it's a reasonable tour.

**Note**: Repeated running of this algorithm produces slightly different routes, varying in length from ~60km to ~66km.

***

## Alternate algorithms

### Exact Solution

#### Brute Force

The initial naive idea when considering this route optimization problem would be to find the exact solution via brute force.  This would involve running every single possible route and saving the shortest one.  The problem is that the number of possible routes given $ N $ nodes goes with $ O(N!) $, meaning that for 50 tot sprays we'd have to consider an infeasible $ >10^{60} $ routes.  For 7 or 8 nodes, brute force might be worth a try but for this problem it's a no-go.

#### Branch and Bound

Another option for producing an exact solution is the Branch and Bound (BnB) method.  This method sets a minimum bound on the possible shortest route.  As the algorithm progresses through the nodes, the minimum bound is continuously updated as the route gets closer to the end.  Once the end is reached, the route is saved as the best route so far and the algorithm starts again.  Now the next node is only chosen if the minimum bound is less than the best route, otherwise that node and all nodes that come after it are pruned.

In an ideal case, the BnB algorithm can significantly reduce the number of routes considered, so I tried implementing it for the tot spray data and still found it too computationally taxing (code available upon request).  I concluded that an exact solution was out of reach for the time being and went on to try approximation algorithms.

### Approximation Algorithms

#### Nearest-Neighbor

On the opposite end of the spectrum is the Nearest-Neighbor, or "greedy", algorithm.  This is a common sense approach in which the driver would start at City Hall, pick the closest tot spray and drive to it.  Then she would pick the closest tot spray to *that* location and drive there next, and so on.  At each node, she picks the nearest non-visited node.  The NN algorithm is quick to implement and can effectively provide an immediate route improvement, but is not the strongest alternative.  It is possible to construct a set of nodes for which NN will give the *worst* route (although that arrangement might be pretty odd).

#### 3-opt or k-opt

If 2-opt, why not 3-opt, or more!  Another possibility to further optimize the route would be to improve the pairwise swapping algorithm presented here by cutting and swapping 2 or more edges, rather than just 1.  The algorithm generates all possible edge combinations for each tour, loops over them, and checks if the tour length improves.  The underlying principle is the same as 2-opt, but the logic is more complex: for 3-opt, each edge swap has four possible route alterations.  Given that complexity, a complete 3-opt algorithm also takes longer to run.

I chose the 2-opt algorithm because the logic is straightforward, the runtime on 50 nodes was manageable, and the output produced a clear route improvement.